In [1]:
import altair as alt
from altair import datum
import pandas as pd
import os 
from vega_datasets import data

# Convert datasets into one

In [133]:
def modifyDataset(temps, GlobalTemps, hdi):
    yearTemps = [[0]*12, [0]*12]
    totalTemps = [[], []]
    GlobalTemps['year'] = pd.to_datetime(GlobalTemps.year)
    for i in range(len(GlobalTemps)):
        monthData = GlobalTemps.iloc[i]
        date = GlobalTemps['year'].iloc[i]
        yearTemps[0][monthData['year'].month-1] = monthData['LandMaxTemperature']
        yearTemps[1][monthData['year'].month-1] = monthData['LandMinTemperature']
        if monthData['year'].month == 12:
            totalTemps[0].append(max(yearTemps[0]))
            totalTemps[1].append(min(yearTemps[1]))
    GlobalTemps = GlobalTemps.drop(GlobalTemps[GlobalTemps['year'].dt.month != 12].index)
    GlobalTemps['year'] = GlobalTemps['year'].apply(lambda x: int(x.strftime('%Y-%m-%d')[0:4]))
    GlobalTemps['LandMaxTemperature'] = totalTemps[0]
    GlobalTemps['LandMinTemperature'] = totalTemps[1]
    
    temps = temps.drop(temps[temps['dt'].str[6] != '6'].index)
    temps = temps.rename(columns={'dt': 'year', 'Country': 'country'})
    temps['year'] = temps['year'].apply(lambda x: int(x[0:4]))
    
    hdi = hdi.rename(columns={'Year': 'year', 'Entity': 'country', 'Historical Index of Human Development (Prados de la Escosura)': 'HDI'})
    return temps, GlobalTemps, hdi

In [134]:
# Pull the data
co2 = pd.read_csv(os.getcwd() + '/Data/CO2.csv', sep=',')
temps = pd.read_csv(os.getcwd() + '/Data/GlobalLandTemperaturesByCountry.csv', sep=',')
hdi = pd.read_csv(os.getcwd() + '/Data/HDI.csv', sep=',')
GlobalTemps = pd.read_csv(os.getcwd() + '/Data/GlobalTemperatures.csv', sep=',')
relevant_columns = ['country', 'year', 'co2', 'AverageTemperature', 'population', 'share_global_co2',
                   'AverageTemperatureUncertainty', 'gdp', 'HDI', 'LandMaxTemperature', 'LandMinTemperature']

# Modify the datasets to match, and eliminates non-June months
temps, GlobalTemps, hdi = modifyDataset(temps, GlobalTemps, hdi);

# Unionise the DataFrames
data = pd.merge(co2, temps, on=['year', 'country'])
data = pd.merge(data, hdi, on=['year', 'country'])
data = pd.merge(data, GlobalTemps, on=['year'])
data = data[relevant_columns]
print(data)
data.to_csv("Final_data.csv")

             country  year      co2  AverageTemperature  population  \
0        Afghanistan  1950    0.084              25.083   7752117.0   
1            Albania  1950    0.297              21.372   1263164.0   
2            Algeria  1950    3.784              32.644   8872250.0   
3             Angola  1950    0.187              18.854   4548021.0   
4          Argentina  1950   29.921               8.908  17037910.0   
...              ...   ...      ...                 ...         ...   
2329          Sweden  1880    2.829              11.546   4561661.0   
2330     Switzerland  1880    1.671              12.757   2824957.0   
2331          Turkey  1880    0.139              19.559  12556805.0   
2332  United Kingdom  1880  297.125              12.136  34733737.0   
2333   United States  1880  198.684              18.939  51403129.0   

      share_global_co2  AverageTemperatureUncertainty           gdp   HDI  \
0                 0.00                          0.238  9.421400e+09  0